## BERTを用いたセンチメント分析

<p style="font-size:14px">
<b>sentiment-analysis(感情分析)</b> は自然言語や記号列で構成されたテキストを分類する技術です。<br>
- 仕組み：自然言語処理（NLP）と機械学習モデルを使い、単語や文脈のパターンを学習して判定する。<br>
- 応用例：商品レビューの満足度分析、SNSでのブランド評価の把握、顧客のフィードバックから改善点を抽出など<br>
- イメージ例:<br>
　「この商品は本当に素晴らしい！」 → ポジティブ<br>
　「期待外れでがっかりした。」 → ネガティブ<br>
　「まあまあでした。」 → ニュートラル<br>

<b>感情分析の仕組み</b>
-テキスト例：「この商品は本当に素晴らしい！」<br>
- 前処理（トークナイズ）、つまり文章を単語やサブワードに分割します。<br>
- 日本語: 「この」「商品」「は」「本当」「に」「素晴らしい」「！」のように分ける。<br>
- 中国語："这个", "商品", "真棒" のように分ける。<br>
- 英語："This", "product", "is", "amazing" のように分ける。<br>

<b> 数値化（ベクトル化）、単語を「数値の並び（ベクトル）」に変換します。</b>

</p>


  - 
◆ 良い（ポジティブ）語彙のランク付け
---
| ランク            | 語彙例                                     | スコア | 説明                     |
|-------------------|--------------------------------------------|--------|--------------------------|
| 強いポジティブ    | excellent, outstanding, amazing, fantastic | 2.0    | 非常に強い肯定的評価     |
| 中程度ポジティブ  | good, nice, happy, satisfied               | 1.0    | 一般的な肯定的評価       |
| 弱いポジティブ    | acceptable, fine, okay                     | 0.5    | 積極的ではないが肯定的   |

◆ 悪い（ネガティブ）語彙のランク付け
| ランク            | 語彙例                                     | スコア | 説明                     |
|-------------------|--------------------------------------------|--------|--------------------------|
| 強いネガティブ    | terrible, awful, horrible, disgusting      | -2.0   | 非常に強い否定的評価     |
| 中程度ネガティブ  | bad, poor, unsatisfied, weak               | -1.0   | 一般的な否定的評価       |
| 弱いネガティブ    | not good, mediocre, disappointing          | -0.5   | 軽度の否定的評価         |

- モデルによる処理（BERTやDistilBERTなど）
- ニューラルネットワークが文脈を理解し、文章全体の特徴を抽出します。
- 「ポジティブっぽい単語が多い」「否定的な表現がある」などを学習済みの重みで判断。
- 分類器でラベルを決定
- 出力された特徴ベクトルを「ポジティブ」「ネガティブ」「ニュートラル」などのラベルに分類。
- 例えば positive: 0.92, negative: 0.05, neutral: 0.03 のような確率スコアが出ます。
- 結果を返す
- 最も高いスコアのラベルを「総合判定」として返す。
- 上記例なら「ポジティブ」と判定。


In [15]:
#!pip install torch transformers
!pip install -q torch transformers

In [1]:
from transformers import AutoModelForSequenceClassification, BertJapaneseTokenizer, pipeline

# 分類モデル（皮肉・感情判定）
model = AutoModelForSequenceClassification.from_pretrained(
    "C:/hf_models/bert-irony",
    local_files_only=True
)

# 日本語BERTトークナイザ
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "C:/hf_models/bert-japanese-tokenizer",
    local_files_only=True
)

# パイプライン作成
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
texts = [
    "修理に出します。購入して１年ちょっとで壊れてしまいました。残念です。",
    "とても素晴らしい商品で大満足です！また買いたいです。",
    "普通でした。可もなく不可もなくといった感じです。",
    "本当にひどい対応でした。二度と利用しません。",
]

results = nlp(texts)

for t, r in zip(texts, results):
    print("テキスト:", t)
    print("ラベル:", r["label"])
    print("スコア:", r["score"])
    print("-" * 40)

#csv形式で保存
import csv, os

save_dir = "C:/temp/"
os.makedirs(save_dir, exist_ok=True)
csv_filename = os.path.join(save_dir, "sentiment_results.csv")

with open(csv_filename, "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["text", "label", "score"])
    for text, result in zip(texts, results):
        writer.writerow([text, result["label"], result["score"]])

print("保存完了:", csv_filename)

C:\Users\mzjin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Device set to use cpu


テキスト: 修理に出します。購入して１年ちょっとで壊れてしまいました。残念です。
ラベル: ポジティブ
スコア: 0.6155224442481995
----------------------------------------
テキスト: とても素晴らしい商品で大満足です！また買いたいです。
ラベル: ネガティブ
スコア: 0.5867560505867004
----------------------------------------
テキスト: 普通でした。可もなく不可もなくといった感じです。
ラベル: ポジティブ
スコア: 0.7164734601974487
----------------------------------------
テキスト: 本当にひどい対応でした。二度と利用しません。
ラベル: ポジティブ
スコア: 0.5232881307601929
----------------------------------------
保存完了: C:/temp/sentiment_results.csv


#### 問題 ! モデル'kit-nlp/bert-base-japanese-sentiment-irony'はおかしいです。
#### ２番目の文はポジティブなのにネガティブと判断しています。４番目の文はネガティブなのにポジティブと判断しています。
#### No1. ポジティブ	0.6155224442481995
#### No2. "とても素晴らしい商品で大満足です！また買いたいです。"	ネガティブ	0.5867560505867004
#### No3. "普通でした。可もなく不可もなくといった感じです。"	ポジティブ	0.7164734601974487
#### No4. "本当にひどい対応でした。二度と利用しません。"	ポジティブ	0.5232881307601929

## モデル"koheiduck/bert-japanese-finetuned-sentiment"を用いたセンチメント分析

In [2]:
from transformers import AutoModelForSequenceClassification, BertJapaneseTokenizer, pipeline

# モデル（koheiduck版の感情分析）
model = AutoModelForSequenceClassification.from_pretrained(
    "C:/hf_models/bert-sentiment-kohei",
    local_files_only=True
)

# トークナイザ（同じフォルダに vocab.txt 等がある）
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "C:/hf_models/bert-sentiment-kohei",
    local_files_only=True
)

# パイプライン作成
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# テスト
texts = [
    "この商品は本当に素晴らしいです！",
    "期待外れでした。二度と買いません。",
    "まあまあでした。普通です。"
]

results = nlp(texts)
for t, r in zip(texts, results):
    print("テキスト:", t)
    print("ラベル:", r["label"])
    print("スコア:", r["score"])
    print("-" * 40)

Device set to use cpu


テキスト: この商品は本当に素晴らしいです！
ラベル: POSITIVE
スコア: 0.9939815402030945
----------------------------------------
テキスト: 期待外れでした。二度と買いません。
ラベル: NEGATIVE
スコア: 0.959332287311554
----------------------------------------
テキスト: まあまあでした。普通です。
ラベル: NEUTRAL
スコア: 0.9439112544059753
----------------------------------------


### 複数の文の確信度を統合した一つの総合値を返す。

In [4]:
import re
from transformers import (
    AutoModelForSequenceClassification,
    BertJapaneseTokenizer,
    pipeline,
)

# モデルとトークナイザ（ローカルキャッシュのみ使用）
model = AutoModelForSequenceClassification.from_pretrained(
    "C:/hf_models/bert-sentiment-kohei",
    local_files_only=True,
)
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "C:/hf_models/bert-sentiment-kohei",
    local_files_only=True,
)

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 複数文テキスト
texts = [
    "修理に出します。購入して１年ちょっとで壊れてしまいました。残念です。",
    "とても素晴らしい商品で大満足です！また買いたいです。",
    "普通でした。可もなく不可もなくといった感じです。",
    "本当にひどい対応でした。二度と利用しません。",
]

# 文ごとに推論
results = nlp(texts)

# ---- スコアの統合 ----
scores = {}
for r in results:
    label = r["label"]
    score = r["score"]
    scores[label] = scores.get(label, 0.0) + score

# 合計スコアが最大のラベル = 総合判定
overall_label = max(scores, key=scores.get)

# ---- 表示 ----
print("◆ 文ごとの結果")
for s, r in zip(texts, results):   # ← ここを texts に修正
    print("文:", s)
    print("  ラベル:", r["label"])
    print("  スコア(確信度):", r["score"])
    print("-" * 30)

print("\n◆ ラベル別 合計スコア")
for label, sc in scores.items():
    print(f"  {label}: {sc}")

print("\n◆ 総合判定:", overall_label)

Device set to use cpu


◆ 文ごとの結果
文: 修理に出します。購入して１年ちょっとで壊れてしまいました。残念です。
  ラベル: NEGATIVE
  スコア(確信度): 0.9835190176963806
------------------------------
文: とても素晴らしい商品で大満足です！また買いたいです。
  ラベル: POSITIVE
  スコア(確信度): 0.9915620684623718
------------------------------
文: 普通でした。可もなく不可もなくといった感じです。
  ラベル: NEUTRAL
  スコア(確信度): 0.9042871594429016
------------------------------
文: 本当にひどい対応でした。二度と利用しません。
  ラベル: NEGATIVE
  スコア(確信度): 0.9896553754806519
------------------------------

◆ ラベル別 合計スコア
  NEGATIVE: 1.9731743931770325
  POSITIVE: 0.9915620684623718
  NEUTRAL: 0.9042871594429016

◆ 総合判定: NEGATIVE


In [19]:
### ニュートラルの項目を加えるコード

In [5]:
import re
from transformers import AutoModelForSequenceClassification, BertJapaneseTokenizer, pipeline

# モデル・トークナイザ（ローカルキャッシュのみ使用）
tokenizer = BertJapaneseTokenizer.from_pretrained(
   # "cl-tohoku/bert-base-japanese-whole-word-masking",
        "C:/hf_models/bert-sentiment-kohei",
    local_files_only=True,
)
model = AutoModelForSequenceClassification.from_pretrained(
    #"koheiduck/bert-japanese-finetuned-sentiment",
        "C:/hf_models/bert-sentiment-kohei",
    local_files_only=True,
)

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# ★ ラベル番号→意味 のマッピング（例）
# 実際は model.config.id2label を print して確認してから調整してください。
raw_id2label = model.config.id2label  # 例: {0: 'LABEL_0', 1: 'LABEL_1', 2:'LABEL_2'}
print("モデル定義のラベル:", raw_id2label)

# ここで「人間用ラベル」にマッピング（例として書いています）
label_map = {
    "LABEL_0": "NEGATIVE",
    "LABEL_1": "NEUTRAL",
    "LABEL_2": "POSITIVE",
}

# ---- 複数文テキスト ----
text = (
    "とても素晴らしい商品で大満足です！また買いたいです。"
    "普通でした。可もなく不可もなくといった感じです。"
    "本当にひどい対応でした。二度と利用しません。"
)

# 文分割（。！!？? でざっくり）
sentences = re.split(r"[。！!？?]", text)
sentences = [s.strip() for s in sentences if s.strip()]

# 文ごとの推論
results = nlp(sentences)

# ---- スコア集計（人間用ラベルで合算）----
scores = {"POSITIVE": 0.0, "NEUTRAL": 0.0, "NEGATIVE": 0.0}

for s, r in zip(sentences, results):
    raw_label = r["label"]                 # 例: 'LABEL_0'
    label = label_map.get(raw_label, raw_label)  # 例: 'NEGATIVE' / 'NEUTRAL' / 'POSITIVE'
    score = r["score"]

    scores[label] += score

    print("文:", s)
    print("  生ラベル:", raw_label)
    print("  マップ後:", label)
    print("  スコア:", score)
    print("-" * 30)

# 合計スコアが最大のものを総合判定に
overall = max(scores, key=scores.get)

print("\n◆ ラベル別 合計スコア")
for k, v in scores.items():
    print(f"  {k}: {v}")

print("\n◆ 総合判定:", overall)


Device set to use cpu


モデル定義のラベル: {0: 'NEUTRAL', 1: 'NEGATIVE', 2: 'POSITIVE'}
文: とても素晴らしい商品で大満足です
  生ラベル: POSITIVE
  マップ後: POSITIVE
  スコア: 0.9814527034759521
------------------------------
文: また買いたいです
  生ラベル: NEUTRAL
  マップ後: NEUTRAL
  スコア: 0.5033612847328186
------------------------------
文: 普通でした
  生ラベル: NEUTRAL
  マップ後: NEUTRAL
  スコア: 0.9016615152359009
------------------------------
文: 可もなく不可もなくといった感じです
  生ラベル: NEUTRAL
  マップ後: NEUTRAL
  スコア: 0.8271836042404175
------------------------------
文: 本当にひどい対応でした
  生ラベル: NEGATIVE
  マップ後: NEGATIVE
  スコア: 0.9930428266525269
------------------------------
文: 二度と利用しません
  生ラベル: NEGATIVE
  マップ後: NEGATIVE
  スコア: 0.6537203192710876
------------------------------

◆ ラベル別 合計スコア
  POSITIVE: 0.9814527034759521
  NEUTRAL: 2.232206404209137
  NEGATIVE: 1.6467631459236145

◆ 総合判定: NEUTRAL


## フォルダに格納されているすべてのファイルを読み込み順次に計算した結果をファイルごとに出力する。

In [6]:
import os
import re
import pandas as pd

from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    BertJapaneseTokenizer,
)
from IPython.display import display  # Notebook 用

# モデルとトークナイザ（ローカルキャッシュのみ使用）
model = AutoModelForSequenceClassification.from_pretrained(
    "C:/hf_models/bert-sentiment-kohei",
    local_files_only=True,
)
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "C:/hf_models/bert-sentiment-kohei",
    local_files_only=True,
)

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# レビューが入っているフォルダ
REVIEWS_DIR = r"C:\Users\mzjin\OneDrive\デスクトップ\Reviews"

# CSV 保存先
OUTPUT_DIR = r"C:/temp"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def split_sentences(text: str):
    """文を「。」「！」「？」「!」「?」などで区切る"""
    text = text.replace("\n", " ")
    parts = re.split(r'(?<=[。！？!?])\s*', text)
    return [p.strip() for p in parts if p.strip()]


def analyze_text(text: str):
    """1レビューを文ごとに解析し、ラベルごとの合計スコアと総合判定を返す"""
    sentences = split_sentences(text)

    if not sentences:
        return {
            "sentences": [],
            "results": [],
            "scores": {},
            "overall": None,
        }

    results = nlp(sentences)

    # ラベルごとに合計スコア（小数点4桁に丸める）
    scores = {}
    for r in results:
        label = str(r["label"])
        score = round(float(r["score"]), 4)
        scores[label] = scores.get(label, 0.0) + score

    scores = {k: round(v, 4) for k, v in scores.items()}
    overall_label = max(scores, key=scores.get) if scores else None

    return {
        "sentences": sentences,
        "results": results,
        "scores": scores,
        "overall": overall_label,
    }


# ------------------- 実 行 -------------------

files = [f for f in os.listdir(REVIEWS_DIR) if f.lower().endswith(".txt")]

if not files:
    print("txt ファイルが見つかりませんでした。")
else:
    summary_rows = []
    all_labels = set()

    for idx, filename in enumerate(sorted(files), start=1):
        file_path = os.path.join(REVIEWS_DIR, filename)

        with open(file_path, "r", encoding="utf-8-sig") as f:
            text = f.read()

        result = analyze_text(text)
        scores = result["scores"]
        all_labels.update(scores.keys())

        # テキスト冒頭5文字を抽出
        head5 = text.strip()[:5]

        # 短いラベル形式: Rev.xx_冒頭5文字
        display_name = f"Rev.{idx:02d}_{head5}"

        row = {
            "filename": display_name,
            "num_sentences": len(result["sentences"]),
            "scores": scores,
            "overall_label": result["overall"],
        }
        summary_rows.append(row)

    # ---- ラベルごとに列を展開 ----
    all_labels = sorted(all_labels)

    expanded_rows = []
    for row in summary_rows:
        base = {
            "filename": row["filename"],
            "num_sentences": row["num_sentences"],
            "overall_label": row["overall_label"],
        }
        for label in all_labels:
            base[f"{label}_score"] = row["scores"].get(label, 0.0)
        expanded_rows.append(base)

    # DataFrame 化 & Notebook 表示
    df = pd.DataFrame(expanded_rows)
    display(df)

    # CSV 出力
    output_csv_path = os.path.join(OUTPUT_DIR, "sentiment_summary.csv")
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

    print("=" * 60)
    print("Saved file as", output_csv_path)

Device set to use cpu


,filename,num_sentences,overall_label,NEGATIVE_score,NEUTRAL_score,POSITIVE_score
0,Rev.01_スペックは,16,NEGATIVE,10.1339,1.7909,1.6388
1,Rev.02_私自身スマ,6,NEUTRAL,1.4579,2.4354,0.8602
2,Rev.03_とても良い,2,POSITIVE,0.0000,0.0000,1.9879
3,Rev.04_機能面、価,1,POSITIVE,0.0000,0.0000,0.9645
4,Rev.05_この価格で,11,NEGATIVE,6.5925,3.5117,0.0000
5,Rev.06_性能自体は,2,NEGATIVE,0.9890,0.8996,0.0000
6,Rev.07_POCO,30,NEUTRAL,5.0117,12.2009,8.0530
7,Rev.08_凯美瑞后排,1,POSITIVE,0.0000,0.0000,0.7643
8,Rev.09_雅阁居然输,1,NEUTRAL,0.0000,0.7292,0.0000
9,Rev.10_B级车凯美,1,NEGATIVE,0.6338,0.0000,0.0000


Saved file as C:/temp\sentiment_summary.csv


多言語 DistilBERTモデルを用いた感情分析の例。

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# モデル（多言語 DistilBERT 感情分析）
model = AutoModelForSequenceClassification.from_pretrained(
    "C:/hf_models/distilbert-multilingual-sentiment",
    local_files_only=True
)

# トークナイザ（同じフォルダに vocab.txt 等がある）
tokenizer = AutoTokenizer.from_pretrained(
    "C:/hf_models/distilbert-multilingual-sentiment",
    local_files_only=True
)

# パイプライン作成
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# テスト
texts = [
    "この商品は本当に素晴らしいです！",
    "期待外れでした。二度と買いません。",
    "It's okay, not great but not terrible.",
    "Je suis très content de ce produit."
]

results = nlp(texts)
for t, r in zip(texts, results):
    print("テキスト:", t)
    print("ラベル:", r["label"])
    print("スコア:", r["score"])
    print("-" * 40)

Device set to use cpu


テキスト: この商品は本当に素晴らしいです！
ラベル: positive
スコア: 0.8667420744895935
----------------------------------------
テキスト: 期待外れでした。二度と買いません。
ラベル: negative
スコア: 0.3807208240032196
----------------------------------------
テキスト: It's okay, not great but not terrible.
ラベル: neutral
スコア: 0.44427844882011414
----------------------------------------
テキスト: Je suis très content de ce produit.
ラベル: positive
スコア: 0.8748921155929565
----------------------------------------


In [8]:
import os
import re
import pandas as pd

from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
from IPython.display import display  # Notebook 用

# モデルとトークナイザ（ローカルキャッシュのみ使用）
MODEL_DIR = "C:/hf_models/distilbert-multilingual-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR,
    local_files_only=True,
)
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_DIR,
    local_files_only=True,
)

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# レビューが入っているフォルダ
REVIEWS_DIR = r"C:\Users\mzjin\OneDrive\デスクトップ\Reviews"

# CSV 保存先
OUTPUT_DIR = r"C:/temp"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def split_sentences(text: str):
    """文を「。」「！」「？」「!」「?」などで区切る"""
    text = text.replace("\n", " ")
    parts = re.split(r'(?<=[。！？!?])\s*', text)
    return [p.strip() for p in parts if p.strip()]


def analyze_text(text: str):
    """1レビューを文ごとに解析し、ラベルごとの合計スコアと総合判定を返す"""
    sentences = split_sentences(text)

    if not sentences:
        return {
            "sentences": [],
            "results": [],
            "scores": {},
            "overall": None,
        }

    results = nlp(sentences)

    scores = {}
    for r in results:
        label = str(r["label"])
        score = round(float(r["score"]), 4)
        scores[label] = scores.get(label, 0.0) + score

    scores = {k: round(v, 4) for k, v in scores.items()}
    overall_label = max(scores, key=scores.get) if scores else None

    return {
        "sentences": sentences,
        "results": results,
        "scores": scores,
        "overall": overall_label,
    }


# ------------------- 実 行 -------------------

files = [f for f in os.listdir(REVIEWS_DIR) if f.lower().endswith(".txt")]

if not files:
    print("txt ファイルが見つかりませんでした。")
else:
    summary_rows = []
    all_labels = set()

    for idx, filename in enumerate(sorted(files), start=1):
        file_path = os.path.join(REVIEWS_DIR, filename)

        with open(file_path, "r", encoding="utf-8-sig") as f:
            text = f.read().strip()

        result = analyze_text(text)
        scores = result["scores"]
        all_labels.update(scores.keys())

        # 冒頭文字の切り取り条件分岐
        if text and text[0].isascii():  # 半角ローマ字などASCIIなら10文字
            head = text[:10]
        else:  # 日本語など全角なら5文字
            head = text[:5]

        # 短いラベル形式: Rev.xx_冒頭文字
        display_name = f"Rev.{idx:02d}_{head}"

        row = {
            "filename": display_name,
            "num_sentences": len(result["sentences"]),
            "scores": scores,
            "overall_label": result["overall"],
        }
        summary_rows.append(row)

    # ---- ラベルごとに列を展開 ----
    all_labels = sorted(all_labels)

    expanded_rows = []
    for row in summary_rows:
        base = {
            "filename": row["filename"],
            "num_sentences": row["num_sentences"],
            "overall_label": row["overall_label"],
        }
        for label in all_labels:
            base[f"{label}_score"] = row["scores"].get(label, 0.0)
        expanded_rows.append(base)

    # DataFrame 化 & Notebook 表示
    df = pd.DataFrame(expanded_rows)
    display(df)

    # CSV 出力
    output_csv_path = os.path.join(OUTPUT_DIR, "sentiment_summary_multilingual.csv")
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

    print("=" * 60)
    print("Saved file as", output_csv_path)

Device set to use cpu


,filename,num_sentences,overall_label,negative_score,neutral_score,positive_score
0,Rev.01_スペックは,16,negative,6.0383,0.0000,1.9723
1,Rev.02_私自身スマ,6,positive,0.9865,0.0000,2.3548
2,Rev.03_とても良い,2,positive,0.0000,0.0000,1.7664
3,Rev.04_機能面、価,1,positive,0.0000,0.0000,0.6442
4,Rev.05_この価格で,11,negative,2.6432,1.1523,2.3126
5,Rev.06_性能自体は,2,negative,0.7874,0.0000,0.3563
6,Rev.07_POCO F7無印の,30,positive,3.3221,0.3635,13.1352
7,Rev.08_凯美瑞后排,1,negative,0.5115,0.0000,0.0000
8,Rev.09_雅阁居然输,1,negative,0.5190,0.0000,0.0000
9,Rev.10_B级车凯美瑞后排空间,1,positive,0.0000,0.0000,0.6347


Saved file as C:/temp\sentiment_summary_multilingual.csv


In [ ]:
modernBERT-base-multilingual-sentiment は、16以上の言語に対応した多言語感情分析モデルで、BERTを改良した ModernBERT を基盤に、製品レビューやSNS投稿などのテキストを「ポジティブ」「ネガティブ」「ニュートラル」に分類できるようにファインチューニングされています。
- 開発元: Hugging Face 上で公開されている clapAI チームのモデル。
- 基盤モデル: answerdotai/ModernBERT-base をベースに構築。
- 学習データ: clapAI/MultiLingualSentiment データセットを用いてファインチューニング。
- 対応言語: 英語、日本語、中国語、韓国語、フランス語、スペイン語、ポルトガル語、ドイツ語、イタリア語、ロシア語、アラビア語など、16以上の主要言語。
 Hugging Face の transformers ライブラリから簡単にロードできます。



### modernBERT-base-multilingual-sentiment ####
#### は、16以上の言語に対応した多言語感情分析モデルで、BERTを改良した ModernBERT を基盤に、製品レビューやSNS投稿などのテキストを「ポジティブ」「ネガティブ」「ニュートラル」に分類できるようにファインチューニングされています。


In [9]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
)
import pandas as pd

# モデルとトークナイザ（ローカルキャッシュのみ使用）
MODEL_DIR = r"C:\hf_models\ModernBERT-multilingual-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR,
    local_files_only=True,
    reference_compile=False,   # ← これがポイント
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_DIR,
    local_files_only=True,
)

clf = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    max_length=512,   # 任意（警告を消したいなら設定）
    top_k=None,       # 全ラベルのスコアを見たい場合
)

texts = [
    "この映画はとても面白かった。",
    "サービスの対応が最悪で二度と使いたくない。",
    "まあまあかな、良くも悪くもない。",
    "이 제품을 정말 좋아해요!",
    "고객 서비스가 정말 실망스러웠어요.",
    "我非常喜欢这款产品！",
    "质量真的很差。"
]

# 推論結果を表形式に整形
records = []
for t, r in zip(texts, clf(texts)):
    row = {"文": t}
    # 各ラベルのスコアを格納
    for item in r:
        row[item["label"]] = round(item["score"], 4)
    # 最もスコアが高いラベルを推定ラベルとして追加
    row["推定ラベル"] = max(r, key=lambda x: x["score"])["label"]
    records.append(row)

# DataFrame化して表示
df = pd.DataFrame(records)
df

Device set to use cpu


,文,positive,neutral,negative,推定ラベル
0,この映画はとても面白かった。,0.8667,0.1298,0.0035,positive
1,サービスの対応が最悪で二度と使いたくない。,0.0000,0.0003,0.9997,negative
2,まあまあかな、良くも悪くもない。,0.0164,0.9812,0.0024,neutral
3,이 제품을 정말 좋아해요!,0.9656,0.0176,0.0168,positive
4,고객 서비스가 정말 실망스러웠어요.,0.0493,0.2854,0.6652,negative
5,我非常喜欢这款产品！,0.9500,0.0268,0.0232,positive
6,质量真的很差。,0.0030,0.0195,0.9776,negative


In [ ]:
### ModernBERTを用いてフォールだに格納したファイルを読み込み感情分析を行う例

In [10]:
import os
import re
import string
import pandas as pd
from IPython.display import display

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
)

# モデルとトークナイザ（ローカルキャッシュのみ使用）
MODEL_DIR = r"C:\hf_models\ModernBERT-multilingual-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR,
    local_files_only=True,
    reference_compile=False,
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_DIR,
    local_files_only=True,
)

clf = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    max_length=512,
    top_k=None,   # 全ラベルのスコアを返す
)

# レビューが入っているフォルダ
REVIEWS_DIR = r"C:\Users\mzjin\OneDrive\デスクトップ\Reviews"

# CSV 保存先
OUTPUT_DIR = r"C:/temp"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def split_sentences(text: str):
    """文を「。」「！」「？」「!」「?」などで区切る"""
    text = text.replace("\n", " ")
    parts = re.split(r'(?<=[。！？!?])\s*', text)
    return [p.strip() for p in parts if p.strip()]


def analyze_text(text: str):
    """1レビューを文ごとに解析し、ラベルごとの合計スコアと総合判定を返す"""
    sentences = split_sentences(text)

    if not sentences:
        return {
            "sentences": [],
            "results": [],
            "scores": {},
            "overall": None,
        }

    results = clf(sentences)

    scores = {}
    for rlist in results:   # 各文の結果はリスト
        for r in rlist:     # 各ラベルのスコアを取り出す
            label = str(r["label"])
            score = round(float(r["score"]), 4)
            scores[label] = scores.get(label, 0.0) + score

    scores = {k: round(v, 4) for k, v in scores.items()}
    overall_label = max(scores, key=scores.get) if scores else None

    return {
        "sentences": sentences,
        "results": results,
        "scores": scores,
        "overall": overall_label,
    }


# ------------------- 実 行 -------------------

files = [f for f in os.listdir(REVIEWS_DIR) if f.lower().endswith(".txt")]

if not files:
    print("txt ファイルが見つかりませんでした。")
else:
    summary_rows = []
    all_labels = set()

    for idx, filename in enumerate(sorted(files), start=1):
        file_path = os.path.join(REVIEWS_DIR, filename)

        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read().strip()

        result = analyze_text(text)
        scores = result["scores"]
        all_labels.update(scores.keys())

        # 冒頭文字の切り取り条件分岐
        if text and text[0] in string.printable:  # ASCII文字なら10文字
            head = text[:10]
        else:  # 日本語など全角なら5文字
            head = text[:5]

        display_name = f"Rev.{idx:02d}_{head}"

        row = {
            "filename": display_name,
            "num_sentences": len(result["sentences"]),
            "scores": scores,
            "overall_label": result["overall"],
        }
        summary_rows.append(row)

    # ---- ラベルごとに列を展開 ----
    all_labels = sorted(all_labels)

    expanded_rows = []
    for row in summary_rows:
        base = {
            "filename": row["filename"],
            "num_sentences": row["num_sentences"],
            "overall_label": row["overall_label"],
        }
        for label in all_labels:
            base[f"{label}_score"] = row["scores"].get(label, 0.0)
        expanded_rows.append(base)

    df = pd.DataFrame(expanded_rows)
    display(df)

    output_csv_path = os.path.join(OUTPUT_DIR, "ModernBERT-multilingual-sentiment.csv")
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

    print("=" * 60)
    print("Saved file as", output_csv_path)

Device set to use cpu


,filename,num_sentences,overall_label,negative_score,neutral_score,positive_score
0,Rev.01_スペックは,16,neutral,4.1168,9.1020,2.7815
1,Rev.02_私自身スマ,6,positive,0.4987,2.0745,3.4271
2,Rev.03_とても良い,2,positive,0.0038,0.0457,1.9505
3,Rev.04_機能面、価,1,positive,0.0101,0.2510,0.7389
4,Rev.05_この価格で,11,neutral,3.8048,4.8026,2.3924
5,Rev.06_性能自体は,2,neutral,0.1457,1.8311,0.0231
6,Rev.07_POCO F7無印の,30,positive,4.2560,12.7092,13.0346
7,Rev.08_凯美瑞后排,1,positive,0.2235,0.2078,0.5686
8,Rev.09_雅阁居然输,1,neutral,0.3428,0.4945,0.1627
9,Rev.10_B级车凯美瑞后排空间,1,positive,0.3089,0.3211,0.3701


Saved file as C:/temp\ModernBERT-multilingual-sentiment.csv


In [ ]:
同じ日本語レビュー文を ModernBERT-multilingual-sentiment と XLM-RoBERTa の両方で感情分析し、結果を比較するコード例 を示します。

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd

# -----------------------------
# 1. ModernBERT-multilingual-sentiment (ローカル)
# -----------------------------
MODEL_DIR_MODERN = r"C:\hf_models\ModernBERT-multilingual-sentiment"

model_modern = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR_MODERN,
    local_files_only=True,
    reference_compile=False,
)
tokenizer_modern = AutoTokenizer.from_pretrained(MODEL_DIR_MODERN, local_files_only=True)

clf_modern = pipeline(
    "sentiment-analysis",
    model=model_modern,
    tokenizer=tokenizer_modern,
    truncation=True,
    max_length=512,
    top_k=None,
)

# -----------------------------
# 2. XLM-RoBERTa (ローカル)
# -----------------------------
MODEL_DIR_XLMR = r"C:\hf_models\XLMR-sentiment"

model_xlmr = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR_XLMR,
    local_files_only=True,   # ← これだけでOK
)

tokenizer_xlmr = AutoTokenizer.from_pretrained(
    MODEL_DIR_XLMR,
    local_files_only=True,
)

clf_xlmr = pipeline(
    "sentiment-analysis",
    model=model_xlmr,
    tokenizer=tokenizer_xlmr,
    truncation=True,
    max_length=512,
    top_k=None,
)

# -----------------------------
# 3. 比較対象の日本語レビュー文
# -----------------------------
texts = [
    "この映画はとても面白かった。",
    "サービスの対応が最悪で二度と使いたくない。",
    "まあまあかな、良くも悪くもない。",
    "この製品は本当に素晴らしいです！",
    "品質が悪くてがっかりしました。",
    "이 제품을 정말 좋아해요!",
    "고객 서비스가 정말 실망스러웠어요.",
    "我非常喜欢这款产品！",
    "质量真的很差。"
]

# -----------------------------
# 4. 推論と結果整形
# -----------------------------
records = []
for t in texts:
    res_modern = clf_modern([t])   # 文をリストで渡す
    res_xlmr = clf_xlmr([t])

    # ModernBERT の結果
    modern_scores = {}
    for rlist in res_modern:   # 各文の結果
        for r in rlist:        # 各ラベルのスコア
            modern_scores[r["label"]] = round(r["score"], 4)
    modern_label = max(modern_scores, key=modern_scores.get)

    # XLM-RoBERTa の結果
    xlmr_scores = {}
    for rlist in res_xlmr:
        for r in rlist:
            xlmr_scores[r["label"]] = round(r["score"], 4)
    xlmr_label = max(xlmr_scores, key=xlmr_scores.get)

    row = {
        "文": t,
        "ModernBERT_label": modern_label,
        "ModernBERT_scores": modern_scores,
        "XLM-RoBERTa_label": xlmr_label,
        "XLM-RoBERTa_scores": xlmr_scores,
    }
    records.append(row)

df = pd.DataFrame(records)
display(df)

Device set to use cpu
Device set to use cpu


,文,ModernBERT_label,ModernBERT_scores,XLM-RoBERTa_label,XLM-RoBERTa_scores
0,この映画はとても面白かった。,positive,"{'positive': 0.8667, 'neutral': 0.1298, 'negat...",positive,"{'positive': 0.9842, 'neutral': 0.0092, 'negat..."
1,サービスの対応が最悪で二度と使いたくない。,negative,"{'negative': 0.9997, 'neutral': 0.0003, 'posit...",negative,"{'negative': 0.9682, 'neutral': 0.0174, 'posit..."
2,まあまあかな、良くも悪くもない。,neutral,"{'neutral': 0.9812, 'positive': 0.0164, 'negat...",positive,"{'positive': 0.7815, 'neutral': 0.183, 'negati..."
3,この製品は本当に素晴らしいです！,positive,"{'positive': 0.9949, 'neutral': 0.0043, 'negat...",positive,"{'positive': 0.9837, 'negative': 0.0085, 'neut..."
4,品質が悪くてがっかりしました。,negative,"{'negative': 0.9402, 'neutral': 0.0591, 'posit...",negative,"{'negative': 0.9686, 'positive': 0.0164, 'neut..."
5,이 제품을 정말 좋아해요!,positive,"{'positive': 0.9656, 'neutral': 0.0176, 'negat...",positive,"{'positive': 0.9867, 'neutral': 0.0086, 'negat..."
6,고객 서비스가 정말 실망스러웠어요.,negative,"{'negative': 0.6652, 'neutral': 0.2854, 'posit...",negative,"{'negative': 0.9439, 'positive': 0.0308, 'neut..."
7,我非常喜欢这款产品！,positive,"{'positive': 0.95, 'neutral': 0.0268, 'negativ...",positive,"{'positive': 0.9841, 'neutral': 0.0102, 'negat..."
8,质量真的很差。,negative,"{'negative': 0.9776, 'neutral': 0.0195, 'posit...",negative,"{'negative': 0.9744, 'neutral': 0.0133, 'posit..."


In [ ]:
### XML-BERTaを用いてフォルダに格納したファイルを読み込み感情分析を行う例。

In [12]:
import os
import re
import string
import pandas as pd
from IPython.display import display

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
)

# -----------------------------
# XLM-RoBERTa (ローカル)
# -----------------------------
MODEL_DIR_XLMR = r"C:\hf_models\XLMR-sentiment"

model_xlmr = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR_XLMR,
    local_files_only=True,   # ← これだけでOK
)

tokenizer_xlmr = AutoTokenizer.from_pretrained(
    MODEL_DIR_XLMR,
    local_files_only=True,
)

clf_xlmr = pipeline(
    "sentiment-analysis",
    model=model_xlmr,
    tokenizer=tokenizer_xlmr,
    truncation=True,
    max_length=512,
    top_k=None,
)


# レビューが入っているフォルダ
REVIEWS_DIR = r"C:\Users\mzjin\OneDrive\デスクトップ\Reviews"

# CSV 保存先
OUTPUT_DIR = r"C:/temp"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def split_sentences(text: str):
    """文を「。」「！」「？」「!」「?」などで区切る"""
    text = text.replace("\n", " ")
    parts = re.split(r'(?<=[。！？!?])\s*', text)
    return [p.strip() for p in parts if p.strip()]


def analyze_text(text: str):
    """1レビューを文ごとに解析し、ラベルごとの合計スコアと総合判定を返す"""
    sentences = split_sentences(text)

    if not sentences:
        return {
            "sentences": [],
            "results": [],
            "scores": {},
            "overall": None,
        }

    results = clf(sentences)

    scores = {}
    for rlist in results:   # 各文の結果はリスト
        for r in rlist:     # 各ラベルのスコアを取り出す
            label = str(r["label"])
            score = round(float(r["score"]), 4)
            scores[label] = scores.get(label, 0.0) + score

    scores = {k: round(v, 4) for k, v in scores.items()}
    overall_label = max(scores, key=scores.get) if scores else None

    return {
        "sentences": sentences,
        "results": results,
        "scores": scores,
        "overall": overall_label,
    }


# ------------------- 実 行 -------------------

files = [f for f in os.listdir(REVIEWS_DIR) if f.lower().endswith(".txt")]

if not files:
    print("txt ファイルが見つかりませんでした。")
else:
    summary_rows = []
    all_labels = set()

    for idx, filename in enumerate(sorted(files), start=1):
        file_path = os.path.join(REVIEWS_DIR, filename)

        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read().strip()

        result = analyze_text(text)
        scores = result["scores"]
        all_labels.update(scores.keys())

        # 冒頭文字の切り取り条件分岐
        if text and text[0] in string.printable:  # ASCII文字なら10文字
            head = text[:10]
        else:  # 日本語など全角なら5文字
            head = text[:5]

        display_name = f"Rev.{idx:02d}_{head}"

        row = {
            "filename": display_name,
            "num_sentences": len(result["sentences"]),
            "scores": scores,
            "overall_label": result["overall"],
        }
        summary_rows.append(row)

    # ---- ラベルごとに列を展開 ----
    all_labels = sorted(all_labels)

    expanded_rows = []
    for row in summary_rows:
        base = {
            "filename": row["filename"],
            "num_sentences": row["num_sentences"],
            "overall_label": row["overall_label"],
        }
        for label in all_labels:
            base[f"{label}_score"] = row["scores"].get(label, 0.0)
        expanded_rows.append(base)

    df = pd.DataFrame(expanded_rows)
    display(df)

    output_csv_path = os.path.join(OUTPUT_DIR, "ModernBERT-multilingual-sentiment.csv")
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

    print("=" * 60)
    print("Saved file as", output_csv_path)

Device set to use cpu


,filename,num_sentences,overall_label,negative_score,neutral_score,positive_score
0,Rev.01_スペックは,16,neutral,4.1168,9.1020,2.7815
1,Rev.02_私自身スマ,6,positive,0.4987,2.0745,3.4271
2,Rev.03_とても良い,2,positive,0.0038,0.0457,1.9505
3,Rev.04_機能面、価,1,positive,0.0101,0.2510,0.7389
4,Rev.05_この価格で,11,neutral,3.8048,4.8026,2.3924
5,Rev.06_性能自体は,2,neutral,0.1457,1.8311,0.0231
6,Rev.07_POCO F7無印の,30,positive,4.2560,12.7092,13.0346
7,Rev.08_凯美瑞后排,1,positive,0.2235,0.2078,0.5686
8,Rev.09_雅阁居然输,1,neutral,0.3428,0.4945,0.1627
9,Rev.10_B级车凯美瑞后排空间,1,positive,0.3089,0.3211,0.3701


Saved file as C:/temp\ModernBERT-multilingual-sentiment.csv


In [ ]:
### 三つの多言語BERTモデルを用いた

In [13]:
import os
import re
import string
import pandas as pd
from IPython.display import display

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
)

# -----------------------------
# 1. distilbert-multilingual-sentiment (ローカル)
# -----------------------------
MODEL_DIR_DISTIL = r"C:\hf_models\distilbert-multilingual-sentiment"

model_distil = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR_DISTIL,
    local_files_only=True,
)
tokenizer_distil = AutoTokenizer.from_pretrained(
    MODEL_DIR_DISTIL,
    local_files_only=True,
)

clf_distil = pipeline(
    "sentiment-analysis",
    model=model_distil,
    tokenizer=tokenizer_distil,
    truncation=True,
    max_length=512,
    top_k=None,  # 全ラベルのスコアを取得
)

# -----------------------------
# 2. ModernBERT-multilingual-sentiment (ローカル)
# -----------------------------
MODEL_DIR_MODERN = r"C:\hf_models\ModernBERT-multilingual-sentiment"

model_modern = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR_MODERN,
    local_files_only=True,
    reference_compile=False,  # WindowsでのInductorエラー回避
)
tokenizer_modern = AutoTokenizer.from_pretrained(
    MODEL_DIR_MODERN,
    local_files_only=True,
)

clf_modern = pipeline(
    "sentiment-analysis",
    model=model_modern,
    tokenizer=tokenizer_modern,
    truncation=True,
    max_length=512,
    top_k=None,
)

# -----------------------------
# 3. XLM-RoBERTa (ローカル)
# -----------------------------
MODEL_DIR_XLMR = r"C:\hf_models\XLMR-sentiment"

model_xlmr = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR_XLMR,
    local_files_only=True,
)
tokenizer_xlmr = AutoTokenizer.from_pretrained(
    MODEL_DIR_XLMR,
    local_files_only=True,
)

clf_xlmr = pipeline(
    "sentiment-analysis",
    model=model_xlmr,
    tokenizer=tokenizer_xlmr,
    truncation=True,
    max_length=512,
    top_k=None,
)

# -----------------------------
# 評価対象のテキストファイルフォルダ & 出力先
# -----------------------------
REVIEWS_DIR = r"C:\Users\mzjin\OneDrive\デスクトップ\Reviews"
OUTPUT_DIR = r"C:\temp"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def split_sentences(text: str):
    """文を「。」「！」「？」「!」「?」などで区切る"""
    text = text.replace("\n", " ")
    parts = re.split(r'(?<=[。！？!?])\s*', text)
    return [p.strip() for p in parts if p.strip()]


def analyze_text(text: str, clf_pipeline):
    """
    1レビューを文ごとに解析し、
    ラベルごとの合計スコアと総合判定を返す
    """
    sentences = split_sentences(text)

    if not sentences:
        return {
            "sentences": [],
            "results": [],
            "scores": {},
            "overall": None,
        }

    # top_k=None の pipeline を想定（各文ごとにラベル一覧）
    results = clf_pipeline(sentences)

    scores = {}
    for rlist in results:   # 各文の結果は [{label, score}, ...] のリスト
        for r in rlist:
            label = str(r["label"])
            score = float(r["score"])
            scores[label] = scores.get(label, 0.0) + score

    # 小数4桁に丸め
    scores = {k: round(v, 4) for k, v in scores.items()}
    overall_label = max(scores, key=scores.get) if scores else None

    return {
        "sentences": sentences,
        "results": results,
        "scores": scores,
        "overall": overall_label,
    }


# ------------------- 実 行 -------------------

files = [f for f in os.listdir(REVIEWS_DIR) if f.lower().endswith(".txt")]

if not files:
    print("txt ファイルが見つかりませんでした。")
else:
    summary_rows = []

    for idx, filename in enumerate(sorted(files), start=1):
        file_path = os.path.join(REVIEWS_DIR, filename)

        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read().strip()

        # 各モデルで解析
        result_distil = analyze_text(text, clf_distil)
        result_modern = analyze_text(text, clf_modern)
        result_xlmr = analyze_text(text, clf_xlmr)

        # 冒頭文字の切り取り条件分岐（表示用）
        if text and text[0] in string.printable:  # ASCII文字なら10文字
            head = text[:10]
        else:  # 日本語など全角なら5文字
            head = text[:5]

        display_name = f"Rev.{idx:02d}_{head}"

        # 1レビュー分の行データ
        row = {
            "filename": display_name,
            "num_sentences": len(result_distil["sentences"]),  # 文数はどのモデルでも同じ
            "overall_distil": result_distil["overall"],
            "overall_modern": result_modern["overall"],
            "overall_xlmr": result_xlmr["overall"],
        }

        # 各モデルのラベル別スコアを列に展開（列名にモデル名プレフィックスを付ける）
        for label, score in result_distil["scores"].items():
            row[f"distil_{label}_score"] = score

        for label, score in result_modern["scores"].items():
            row[f"modern_{label}_score"] = score

        for label, score in result_xlmr["scores"].items():
            row[f"xlmr_{label}_score"] = score

        summary_rows.append(row)

    # DataFrame 化
    df = pd.DataFrame(summary_rows)
    display(df)

    # CSV 出力
    output_csv_path = os.path.join(OUTPUT_DIR, "multilingual_sentiment_3models.csv")
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

    print("=" * 60)
    print("Saved file as", output_csv_path)


Device set to use cpu
Device set to use cpu
Device set to use cpu


,filename,num_sentences,overall_distil,overall_modern,overall_xlmr,distil_positive_score,distil_neutral_score,distil_negative_score,modern_neutral_score,modern_positive_score,modern_negative_score,xlmr_positive_score,xlmr_neutral_score,xlmr_negative_score
0,Rev.01_スペックは,16,negative,neutral,negative,4.7422,4.4519,6.8060,9.1019,2.7814,4.1167,4.1273,3.7253,8.1474
1,Rev.02_私自身スマ,6,positive,positive,positive,2.7885,1.2130,1.9985,2.0745,3.4270,0.4986,3.7008,0.9392,1.3600
2,Rev.03_とても良い,2,positive,positive,positive,1.7664,0.1657,0.0680,0.0457,1.9505,0.0038,1.9566,0.0264,0.0170
3,Rev.04_機能面、価,1,positive,positive,positive,0.6442,0.2102,0.1456,0.2510,0.7389,0.0101,0.8221,0.1276,0.0504
4,Rev.05_この価格で,11,negative,neutral,negative,3.8534,3.2501,3.8965,4.8027,2.3925,3.8048,2.8272,2.4403,5.7325
5,Rev.06_性能自体は,2,negative,neutral,negative,0.4870,0.4146,1.0984,1.8311,0.0231,0.1457,0.0575,0.2012,1.7413
6,Rev.07_POCO F7無印の,30,positive,positive,positive,14.7647,7.0282,8.2071,12.7091,13.0349,4.2561,13.4868,11.7978,4.7153
7,Rev.08_凯美瑞后排,1,negative,positive,positive,0.2170,0.2715,0.5115,0.2078,0.5686,0.2235,0.9147,0.0628,0.0224
8,Rev.09_雅阁居然输,1,negative,neutral,negative,0.2197,0.2613,0.5190,0.4945,0.1627,0.3428,0.0084,0.0574,0.9341
9,Rev.10_B级车凯美瑞后排空间,1,positive,positive,neutral,0.6347,0.2121,0.1532,0.3211,0.3701,0.3089,0.0189,0.9772,0.0039


Saved file as C:\temp\multilingual_sentiment_3models.csv


In [ ]:
#### 日本語に特化したに分類のモデル/bert-sentiment-kohei

In [14]:
import os
import re
import pandas as pd

from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    BertJapaneseTokenizer,
)
from IPython.display import display  # Notebook 用

# モデルとトークナイザ（ローカルキャッシュのみ使用）
model = AutoModelForSequenceClassification.from_pretrained(
    "C:/hf_models/bert-sentiment-kohei",
    local_files_only=True,
)
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "C:/hf_models/bert-sentiment-kohei",
    local_files_only=True,
)

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# レビューが入っているフォルダ
REVIEWS_DIR = r"C:\Users\mzjin\OneDrive\デスクトップ\Reviews"

# CSV 保存先
OUTPUT_DIR = r"C:/temp"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def split_sentences(text: str):
    """文を「。」「！」「？」「!」「?」などで区切る"""
    text = text.replace("\n", " ")
    parts = re.split(r'(?<=[。！？!?])\s*', text)
    return [p.strip() for p in parts if p.strip()]


def analyze_text(text: str):
    """1レビューを文ごとに解析し、ラベルごとの合計スコアと総合判定を返す"""
    sentences = split_sentences(text)

    if not sentences:
        return {
            "sentences": [],
            "results": [],
            "scores": {},
            "overall": None,
        }

    results = nlp(sentences)

    scores = {}
    for r in results:
        label = str(r["label"])
        score = round(float(r["score"]), 4)
        scores[label] = scores.get(label, 0.0) + score

    scores = {k: round(v, 4) for k, v in scores.items()}
    overall_label = max(scores, key=scores.get) if scores else None

    return {
        "sentences": sentences,
        "results": results,
        "scores": scores,
        "overall": overall_label,
    }


# ------------------- 実 行 -------------------

files = [f for f in os.listdir(REVIEWS_DIR) if f.lower().endswith(".txt")]

if not files:
    print("txt ファイルが見つかりませんでした。")
else:
    summary_rows = []
    all_labels = set()

    for idx, filename in enumerate(sorted(files), start=1):
        file_path = os.path.join(REVIEWS_DIR, filename)

        with open(file_path, "r", encoding="utf-8-sig") as f:
            text = f.read().strip()

        result = analyze_text(text)
        scores = result["scores"]
        all_labels.update(scores.keys())

        # 冒頭文字の切り取り条件分岐
        if text and text[0].isascii():  # 半角ローマ字などASCIIなら10文字
            head = text[:12]
        else:  # 日本語など全角なら5文字
            head = text[:6]

        # 短いラベル形式: Rev.xx_冒頭文字
        display_name = f"Rev.{idx:02d}_{head}"

        row = {
            "filename": display_name,
            "num_sentences": len(result["sentences"]),
            "scores": scores,
            "overall_label": result["overall"],
        }
        summary_rows.append(row)

    # ---- ラベルごとに列を展開 ----
    all_labels = sorted(all_labels)

    expanded_rows = []
    for row in summary_rows:
        base = {
            "filename": row["filename"],
            "num_sentences": row["num_sentences"],
            "overall_label": row["overall_label"],
        }
        for label in all_labels:
            base[f"{label}_score"] = row["scores"].get(label, 0.0)
        expanded_rows.append(base)

    # DataFrame 化 & Notebook 表示
    df = pd.DataFrame(expanded_rows)
    display(df)

    # CSV 出力
    output_csv_path = os.path.join(OUTPUT_DIR, "sentiment_summary.csv")
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

    print("=" * 60)
    print("Saved file as", output_csv_path)


Device set to use cpu


,filename,num_sentences,overall_label,NEGATIVE_score,NEUTRAL_score,POSITIVE_score
0,Rev.01_スペックは正,16,NEGATIVE,10.1339,1.7909,1.6388
1,Rev.02_私自身スマホ,6,NEUTRAL,1.4579,2.4354,0.8602
2,Rev.03_とても良い商,2,POSITIVE,0.0000,0.0000,1.9879
3,Rev.04_機能面、価格,1,POSITIVE,0.0000,0.0000,0.9645
4,Rev.05_この価格で、,11,NEGATIVE,6.5925,3.5117,0.0000
5,Rev.06_性能自体はい,2,NEGATIVE,0.9890,0.8996,0.0000
6,Rev.07_POCO F7無印の購入,30,NEUTRAL,5.0117,12.2009,8.0530
7,Rev.08_凯美瑞后排空,1,POSITIVE,0.0000,0.0000,0.7643
8,Rev.09_雅阁居然输在,1,NEUTRAL,0.0000,0.7292,0.0000
9,Rev.10_B级车凯美瑞后排空间垫底,1,NEGATIVE,0.6338,0.0000,0.0000


Saved file as C:/temp\sentiment_summary.csv


In [ ]:
#### 日本語に特化したに分類のモデル"C:\hf_models\jarvisx17-sentiment"

In [ ]:
import os
import re
import pandas as pd

from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
from IPython.display import display  # Notebook 用

# モデルとトークナイザ（ローカルキャッシュのみ使用）
MODEL_DIR = "C:/hf_models/jarvisx17-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR,
    local_files_only=True,
)
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_DIR,
    local_files_only=True,
)

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# レビューが入っているフォルダ
REVIEWS_DIR = r"C:\Users\mzjin\OneDrive\デスクトップ\Reviews"

# CSV 保存先
OUTPUT_DIR = r"C:/temp"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def split_sentences(text: str):
    """文を「。」「！」「？」「!」「?」などで区切る"""
    text = text.replace("\n", " ")
    parts = re.split(r'(?<=[。！？!?])\s*', text)
    return [p.strip() for p in parts if p.strip()]


def analyze_text(text: str):
    """1レビューを文ごとに解析し、ラベルごとの合計スコアと総合判定を返す"""
    sentences = split_sentences(text)

    if not sentences:
        return {
            "sentences": [],
            "results": [],
            "scores": {},
            "overall": None,
        }

    results = nlp(sentences)

    scores = {}
    for r in results:
        label = str(r["label"])
        score = round(float(r["score"]), 4)
        scores[label] = scores.get(label, 0.0) + score

    scores = {k: round(v, 4) for k, v in scores.items()}
    overall_label = max(scores, key=scores.get) if scores else None

    return {
        "sentences": sentences,
        "results": results,
        "scores": scores,
        "overall": overall_label,
    }


# ------------------- 実 行 -------------------

files = [f for f in os.listdir(REVIEWS_DIR) if f.lower().endswith(".txt")]

if not files:
    print("txt ファイルが見つかりませんでした。")
else:
    summary_rows = []
    all_labels = set()

    for idx, filename in enumerate(sorted(files), start=1):
        file_path = os.path.join(REVIEWS_DIR, filename)

        with open(file_path, "r", encoding="utf-8-sig") as f:
            text = f.read().strip()

        result = analyze_text(text)
        scores = result["scores"]
        all_labels.update(scores.keys())

        # 冒頭文字の切り取り条件分岐
        if text and text[0].isascii():  # 半角ローマ字などASCIIなら10文字
            head = text[:10]
        else:  # 日本語など全角なら5文字
            head = text[:5]

        # 短いラベル形式: Rev.xx_冒頭文字
        display_name = f"Rev.{idx:02d}_{head}"

        row = {
            "filename": display_name,
            "num_sentences": len(result["sentences"]),
            "scores": scores,
            "overall_label": result["overall"],
        }
        summary_rows.append(row)

    # ---- ラベルごとに列を展開 ----
    all_labels = sorted(all_labels)

    expanded_rows = []
    for row in summary_rows:
        base = {
            "filename": row["filename"],
            "num_sentences": row["num_sentences"],
            "overall_label": row["overall_label"],
        }
        for label in all_labels:
            base[f"{label}_score"] = row["scores"].get(label, 0.0)
        expanded_rows.append(base)

    # DataFrame 化 & Notebook 表示
    df = pd.DataFrame(expanded_rows)
    display(df)

    # CSV 出力
    output_csv_path = os.path.join(OUTPUT_DIR, "jarvisx17-sentiment.csv")
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

    print("=" * 60)
    print("Saved file as", output_csv_path)